# ボートレースのレース情報をクロールしpickleファイルに保存

In [1]:
from datetime import datetime
from datetime import timedelta
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import urllib.request
import time
import pandas as pd
import re
from tqdm.notebook import tqdm

## 1. モジュールをロード
### 1.1 web pageから情報を取ってきてpandas dfに格納するモジュール

#### 1.1.3 レース結果のページからスクレイプするモジュール
- 例：https://boatrace.jp/owpc/pc/race/raceresult?rno=11&jcd=15&hd=20210224

In [2]:
import numpy as np
import re
def scrape_beforeinfo(soup, rno, jcd, hd):   
    # 展示競争のコース・スタートタイムをクロール
    race_result_dict_list_2 = []
    exhibition_start_rows = soup.find(class_="is-w238").find(class_="is-p10-0").find_all("tr")
    
    for i, exhibition_start_row in enumerate(exhibition_start_rows, 1):
        race_result_dict_2 = {"date": "-".join([hd[0:4], hd[5:7], hd[8:10]]),
                        "venue": jcd,
                        "raceNumber": rno[:-1]
                        }

        race_result_dict_2["exhibition_cource"] = i
        race_result_dict_2["枠"] = int(exhibition_start_row.find(class_=re.compile("table1_boatImage1Number")).text)
        race_result_dict_2["exhibition_start_time"] = exhibition_start_row.find(class_="table1_boatImage1Time").text
        
        race_result_dict_list_2.append(race_result_dict_2)
    
    # dictを入れたlistをdfに変換
    beforeinfo_df = pd.DataFrame.from_dict(race_result_dict_list_2)
    
    time.sleep(0.1)

    return beforeinfo_df

### 1.2 そのほかcrawl, scrapeに必要なモジュール

In [3]:
def make_url(crawl_key, rno, jcd, hd):
    """
    :param crawl_key: 何をcrawleするか。選択肢は、"odds3t"（オッズ）, "racelist"(出走表）,
    "beforeinfo" (直前情報）もしくは"raceresult" (レース結果)
    :param rno: レース番号。8Rなど、1-12の数字 + R をstrで
    :param jcd: 会場名。"桐　生"、"びわこ"など
    :param hd: holding day (レース開催日)、2019/03/28などyyyy/mm/ddの形で入力（strで）
    :return dds_url: 公式サイト最終オッズが書かれているページのurl. これを使ってcrawlする
    """
    jcd_dict =  {"桐　生": "01", "戸　田": "02", "江戸川": "03", "平和島": "04", "多摩川": "05", "浜名湖": "06", "蒲　郡": "07", "常　滑": "08",
                "　津　": "09", "三　国": "10", "びわこ": "11", "住之江": "12", "尼　崎": "13", "鳴　門": "14", "丸　亀": "15", "児　島": "16",
                "宮　島": "17", "徳　山": "18", "下　関": "19", "若　松": "20", "芦　屋": "21", "福　岡": "22", "唐　津": "23", "大　村": "24"
                }
    rno = rno[:-1]
    hd = hd[0:4] + hd[5:7] + hd[8:10]

    odds_url = "http://boatrace.jp/owpc/pc/race/" + crawl_key + "?rno=" + rno + "&jcd=" + jcd_dict[jcd] + "&hd=" + hd

    return odds_url


def html_parser(site_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
    }

    try:
        request = urllib.request.Request(url=site_url, headers=headers)
        response = urllib.request.urlopen(request)

        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')

    # データベース作成の際、remotedisconnectedになった場合,そのレースをパス
    except RemoteDisconnected:
        print("remote disconnected error !")
        return None

    except ConnectionResetError:
        print("Connection Reset error !")
        return None

    return soup

## 2. 実行
- 最初の以下の行にてクロールを行う日付を指定

　　　　　　　　`　hd_list = ["2021/02/0" + str(day) for day in range(1,10)]`
- クロール元：ボートレース 公式サイト（https://boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=01&hd=20210325など）
- 保存先：'./crawledData/　以下。日にちごとにファイルを作成し保存

In [4]:
import glob
import os
race_file_set = set([os.path.basename(file) for file in glob.glob(os.path.join('../../../data/crawledData', '*.pkl'))])
start_time_file_set = set([os.path.basename(file) for file in glob.glob(os.path.join('../../../data/crawledData/exhibition_mod', '*.pkl'))])

diff_file_name_list = list(race_file_set - start_time_file_set)
hd_list = [filename[0:4] + "/" + filename[4:6] + "/" + filename[6:8] for filename in diff_file_name_list]
print("{0}日分のデータが未収集".format(len(hd_list)))

99日分のデータが未収集


In [ ]:
jcd_list =  ["桐　生", "戸　田", "江戸川", "平和島", "多摩川", "浜名湖", "蒲　郡", "常　滑",
                "　津　", "三　国", "びわこ", "住之江", "尼　崎", "鳴　門", "丸　亀", "児　島",
                "宮　島", "徳　山", "下　関", "若　松", "芦　屋", "福　岡", "唐　津", "大　村"
            ]

for hd in hd_list:
    print("{0} のデータをクロール中".format(hd))

    # 1日単位でデータを集めてファイルに保存する
    today_race_df_list = []

    for jcd in tqdm(jcd_list):
        for i in range(1, 13):
            rno = str(i) + "R"

            # その日レースがない場所は飛ばすためのtry-except         
            try:
                # 色々なkeyに対してクロールして特定のレースの情報がまとまったdfを作る
                raceResult_url = make_url("beforeinfo", rno, jcd, hd)

                # パース
                soup = html_parser(raceResult_url)

                # 対象サイトをcrawl
                race_information_df = scrape_beforeinfo(soup, rno, jcd, hd)
                race_information_df = race_information_df.set_index(["date", "venue", "raceNumber", "枠"])

                # 今回のレースのデータを本日のデータを集めたリストに格納
                today_race_df_list.append(race_information_df)

            except IndexError:
                # print(hd + " " + jcd + rno +"データなし")
                pass
            except AttributeError:
                pass

    # 本日のレースデータを集めたリストをdfに変換
    today_race_df = pd.concat(today_race_df_list, axis = 0)

    # pickleファイルで保存
    today_race_df.to_pickle('../../../data/crawledData/exhibition_mod/{0}.pkl'.format("".join(hd.split("/"))))

2020/12/30 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2021/01/24 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2021/01/12 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

2020/12/23 のデータをクロール中


  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
# ファイル内容確認用
df = pd.read_pickle('../../../data/crawledData/exhibition_mod/20201124.pkl')
df

In [ ]:
# crawlerの動きを確認する用

crawl_key = "raceresult"
jcd =  "大　村"
hd = "2021/03/31"
rno = "9R"

raceResult_url = make_url(crawl_key, rno, jcd, hd)
print(raceResult_url)

# パース
soup = html_parser(raceResult_url)

# 対象サイトをcrawl
race_information_df = scrape_raceresult(soup, rno, jcd, hd)
race_information_df = race_information_df.set_index(["date", "venue", "raceNumber", "枠"])
race_information_df

In [ ]:
race_information_df[1]